<h1>2. Análisis de Opiniones sobre películas

**2.0 Importaciones necesarias**

Antes de comenzar con el desarrollo del problema, se importan librerías y módulos necesarios.

In [5]:
import urllib
import pandas as pd
import re, time
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer, word_tokenize
from nltk.stem.porter import PorterStemmer
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import random
import matplotlib.pyplot as plt

**2.1 Descripción de datos**

Se dispone de dos dataframes, uno de entrenamiento y otro de prueba. Cada uno de ellos posee 3554 registros. A su vez, cada registro es descrito por dos características: *sentimiento*, cuyo valor puede ser +1 (opinión positiva) ó -1 (opinión negativa) y *texto*, que contiene la opinión del espectador.

In [8]:
#Se leen archivos de entrada que contiene datos de entrenamiento
ftr = open("polarity.train", "r")
#Se leen archivos de entrada que contiene datos de prueba
fts = open("polarity.dev", "r")

#Se crea dataframe para datos de entrenamiento
rows = [line.split(" ",1) for line in ftr.readlines()]
train_df = pd.DataFrame(rows, columns=['Sentiment', 'Text'])
train_df['Sentiment'] = train_df['Sentiment'].convert_objects(convert_numeric=True)

#Se crea dataframe para datos de prueba
rows = [line.split(" ",1) for line in fts.readlines()]
test_df = pd.DataFrame(rows, columns=['Sentiment', 'Text'])
test_df['Sentiment'] = test_df['Sentiment'].convert_objects(convert_numeric=True)
 
#Cantidad de registros por cada set de datos
num_train = train_df.shape[0]
num_test = test_df.shape[0]

**2.2 Preprocesamiento de texto: Stemming**

A continuación, se crea la función *word_extractor* para obtener tokens de un texto (con y sin stemming). Notar que la función recibe como parámetros el texto a analizar, junto con la opción (*True*) o no (*False*) de realizar stemming. 

In [12]:
def word_extractor(text, stemming):
    #Se utiliza algoritmo de Porter para stemming
    stemmer = PorterStemmer()
    #Se obtienen stopwords del idioma ingles
    commonwords = stopwords.words('english')
    text = re.sub(r'([a-z])\1+', r'\1\1', text)
    words = ""

    if stemming:
        #Se realiza lower-casing y stemming
        wordtokens = [stemmer.stem(word.lower()) \
                 for word in word_tokenize(text.decode('utf-8', 'ignore'))]
    else:
        #Se realiza lower-casing, pero no stemming
        wordtokens = [word.lower() for word in word_tokenize(text.decode('utf-8', 'ignore'))]

    #Se eliminan tokens pertenecientes al conjunto de stopwords
    for word in wordtokens:
        if word not in commonwords:
            words += " " + word
    
    return words

Así, se listan los resultados obtenidos utilizando la función *word_extractor* sobre las frases de ejemplo entregadas en el enunciado. Primero se trabaja con stemming, haciendo uso del algoritmo de Porter:

**I love to eat cake**: love eat cake  
**I love eating cake**: love eat cake  
**I loved eating the cake**: love eat cake  
**I do not love eating cake**: love eat cake  
**I don’t love eating the cake**: n’t love eat cake  
**If it’s so simple, why haven’t you done it already?**: ’s simpl , whi n’t done alreadi  
**If you’re good at something, never do it for free**: ’re good someth , never free  
**Well today I found out what Batman can’t do**: well today found batman ca n’t  

Se obtienen resultados pobres. Ello se manifiesta principalmente en que para las cuatro primeras expresiones, el resultado final es el mismo, aún cuando algunas de ellas tienen significados opuestos entre sí. Además, muchas de las expresiones generadas contienen tokens que no tiene significado alguno y se reducen tokens que no son verbos, como *something* y *already*.

Sin el uso de stemming, los resultados son los siguientes:

**I love to eat cake**: love eat cake  
**I love eating cake**: love eating cake  
**I loved eating the cake**: love eating cake  
**I do not love eating cake**: love eating cake  
**I don’t love eating the cake**: n’t love eating cake  
**If it’s so simple, why haven’t you done it already?**: ’s simple , n’t done already  
**If you’re good at something, never do it for free**: ’re good something , never free  
**Well today I found out what Batman can’t do**: well today found batman ca n’t  

Se observa que los verbos no son reducidos a su tronco léxico base, pero ello favorece que no se reduzcan tokens que no corresponde reducir, valga la redundancia. Sin embargo, sigue siendo imposible distinguir entre una frase determinada y su negación. De todas maneras, se aprecia una mayor coherencia en las expresiones resultantes.

**2.3 Preprocesamiento de texto: Lematización**

A continuación, se crea la función *word_extractor2* para obtener tokens de un texto por medio del proceso de lematización.

In [13]:
def word_extractor2(text, sw):
    wordlemmatizer = WordNetLemmatizer()
    #Se obtienen stopwords del idioma ingles
    commonwords = stopwords.words('english')
    text = re.sub(r'([a-z])\1+', r'\1\1', text)
    words = ""
    #Se realiza lower-casing y lematizacion
    wordtokens = [wordlemmatizer.lemmatize(word.lower()) \
             for word in word_tokenize(text.decode('utf-8', 'ignore'))]
    
    #Se eliminan tokens pertenecientes al conjunto de stopwords, en caso de que sw == True
    if sw == True:
        for word in wordtokens:
            if word not in commonwords:
                words += " " + word
    else:
        for word in wordtokens:
            words += " " + word	

    return words

Considerando las mismas expresiones de la secci ́on anterior, se obtienen los siguientes resultados:

**I love to eat cake**: love eat cake  
**I love eating cake**: love eating cake  
**I loved eating the cake**: loved eating cake  
**I do not love eating cake**: love eating cake  
**I don’t love eating the cake**: n’t love eating cake  
**If it’s so simple, why haven’t you done it already?**: ’s simple , n’t done already  
**If you’re good at something, never do it for free**: ’re good something , never free  
**Well today I found out what Batman can’t do**: well today found batman ca n’t  

Los resultados son practicamente idénticos a los obtenidos sin realizar stemming, excepto por la tercera expresión, donde la forma verbal *loved* se mantiene, pero no marca una diferencia significativa.

En resumen, se puede ver que, aunque tanto stemming como lematización tienen sus ventajas y desventajas, lematización obtiene mejores resultados, o en su defecto, la no utilización de stemming.

**2.4 Construcción de vocabulario**

Se procede a generar una representación vectorial para cada una de las instancias de la variable *texto* presentes tanto en el dataset de entrenamiento como en el de prueba. Para ello, se utiliza la función *word_extractor2*, junto con la opción de filtrar stopwords.

In [19]:
#Se genera representación vectorial de la variable texto para datos de entrenamiento
texts_train1 = [word_extractor2(text, True) for text in train_df.Text]
#Se genera representación vectorial de la variable texto para datos de prueba
texts_test1 = [word_extractor2(text, True) for text in test_df.Text]

vectorizer1 = CountVectorizer(ngram_range=(1,1), binary=False)
vectorizer1.fit(np.asarray(texts_train1))
features_train1 = vectorizer1.transform(texts_train1)
features_test1 = vectorizer1.transform(texts_test1)

A partir de estas representaciones vectoriales, se obtiene el vocabulario. 

In [45]:
labels_train = np.asarray((train_df.Sentiment.astype(float)+1)/2.0)
labels_test = np.asarray((test_df.Sentiment.astype(float)+1)/2.0)
vocabulario = vectorizer1.get_feature_names()
dist = list(np.array(features_train1.sum(axis=0)).reshape(-1,))

#Se determina la frecuencia de cada token en el vocabulario
word_freq = zip(vocabulario, dist)
#Se ordenan los tokens por su frecuencia en orden descendente
word_freq_ordered = reversed(sorted(word_freq, key=lambda tup: tup[1]))
positions = 10

#Se imprime top 10 de tokens (esto es, los 10 tokens mas frecuentes en el vocabulario)
print ('Luego, considerando tanto los datos de entrenamiento como los de prueba, es posible formar un \nvocabulario compuesto de 9811 tokens. A partir de este, se crea un top ten, de acuerdo a la \nfrecuencia de cada uno A continuación, se muestra cada token perteneciente a este ranking, \nacompañado de su correspondiente frecuencia:\n')
for tag, count in word_freq_ordered:
    if (positions > 0):
        print tag,':', count, 'ocurrencias'
        positions -= 1
    else:
        break

Luego, considerando tanto los datos de entrenamiento como los de prueba, es posible formar un 
vocabulario compuesto de 9811 tokens. A partir de este, se crea un top ten, de acuerdo a la 
frecuencia de cada uno A continuación, se muestra cada token perteneciente a este ranking, 
acompañado de su correspondiente frecuencia:

film : 583 ocurrencias
movie : 503 ocurrencias
one : 259 ocurrencias
like : 255 ocurrencias
ha : 235 ocurrencias
make : 186 ocurrencias
story : 177 ocurrencias
character : 165 ocurrencias
good : 154 ocurrencias
time : 148 ocurrencias


**2.5 Desempeño de clasificadores**

En ésta sección, se muestra el desempeño de diversos modelos de clasificación aplicados sobre los datos provistos. Para esto, se expondrá un reporte por cada modelo. Dicho reporte indica la precisión, el recall, el valor-F (también conocido como F1-score) y el soporte de cada clase. El reporte es generado por la función *score_model*, que se implementa a continuación:

In [58]:
#Se construye funcion score_model, la cual evalua el desempeno de un determinado clasificador
def score_model(model, x, y, xt, yt, text):
    acc_train = model.score(x,y)
    acc_test = model.score(xt[:-1], yt[:-1])
    print 'Precisión datos de entrenamiento %s: %f'%(text, acc_train)
    print 'Precisión datos de prueba %s: %f'%(text, acc_test)
    print 'Análisis detallado de resultados sobre set de prueba:'
    print (classification_report(yt, model.predict(xt), target_names = ['clase +1', 'clase -1']))

**2.5.1 Clasificador Bayesiano Ingenuo Binario**

Primeramente, se implementa la función *NAIVE_BAYES*, que servirá para el entrenamiento/ajuste de un clasificador Bayesiano Binario.

In [56]:
#Implementacion clasificador bayesiano ingenuo binario
def NAIVE_BAYES(x, y, xt, yt):
    model = BernoulliNB()
    model = model.fit(x, y)
    score_model(model, x, y, xt, yt, 'BernoulliNB')
    return model

De esta manera, se estudian los siguientes casos:

**2.5.1.1 Caso 1: Filtrando stopwords y usando lematización**

In [60]:
#caso 1: filtrando stopwords, con lematizacion
model1 = NAIVE_BAYES(features_train1, labels_train, features_test1, labels_test)

Precisión datos de entrenamiento BernoulliNB: 0.958638
Precisión datos de prueba BernoulliNB: 0.738531
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

   clase +1       0.75      0.73      0.74      1803
   clase -1       0.73      0.75      0.74      1751

avg / total       0.74      0.74      0.74      3554



**2.5.1.2 Caso 2: Sin filtrar stopwords y usando lematización**

Se requiere generar una nueva representación vectorial de la variable *texto*, dadas las características especiales del caso estudiado. 

In [62]:
texts_train2 = [word_extractor2(text, False) for text in train_df.Text]
texts_test2 = [word_extractor2(text, False) for text in test_df.Text]
vectorizer2 = CountVectorizer(ngram_range=(1,1), binary=False)
vectorizer2.fit(np.asarray(texts_train2))
features_train2 = vectorizer2.transform(texts_train2)
features_test2 = vectorizer2.transform(texts_test2)

Con lo anterior, se está en condiciones de construir el modelo.

In [63]:
#caso 2: sin filtrar stopwords, con lematizacion
model2 = NAIVE_BAYES(features_train2, labels_train, features_test2, labels_test)

Precisión datos de entrenamiento BernoulliNB: 0.955262
Precisión datos de prueba BernoulliNB: 0.748663
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

   clase +1       0.76      0.74      0.75      1803
   clase -1       0.74      0.76      0.75      1751

avg / total       0.75      0.75      0.75      3554



**2.5.1.3 Caso 3: Filtrando stopwords y usando stemming**

Al igual que en la sección anterior, es necesario generar una nueva representación vectorial de la variable *texto*, dadas las características especiales del caso estudiado.

In [65]:
texts_train3 = [word_extractor(text, True) for text in train_df.Text]
texts_test3 = [word_extractor(text, True) for text in test_df.Text]
vectorizer3 = CountVectorizer(ngram_range=(1,1), binary=False)
vectorizer3.fit(np.asarray(texts_train3))
features_train3 = vectorizer3.transform(texts_train3)
features_test3 = vectorizer3.transform(texts_test3)

Con lo anterior, se está en condiciones de construir el modelo.

In [66]:
#caso 3: Filtrando stopwords, con stemming
model3 = NAIVE_BAYES(features_train3, labels_train, features_test3, labels_test)

Precisión datos de entrenamiento BernoulliNB: 0.942881
Precisión datos de prueba BernoulliNB: 0.747819
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

   clase +1       0.76      0.74      0.75      1803
   clase -1       0.74      0.75      0.75      1751

avg / total       0.75      0.75      0.75      3554



Así, se obtiene una mayor precisión sobre el set de entrenamiento al usar lematización, pero la precisión sobre el set de prueba es mayor con stemming. Si se toman en cuenta la precisión y recall por cada clase (en cada caso), puede decirse que los mejores resultados se consiguen al usar lematización y no filtrar stopwords.

**2.5.1.4 Análisis de predicciones**

Se han tomado cinco textos y se muestra la predicción sobre cada uno. Sólo se considera el caso 1, dado a que es el modelo que obtiene los mejores resultados (NOTA: Dado a que el siguiente código escoge los textos en forma azarosa, los resultados que imprime no coincidirán con los ejemplos analizados más adelante).

In [70]:
test_pred1 = model1.predict_proba(features_test1)
spl1 = random.sample(xrange(len(test_pred1)), 5)
for text, sentiment in zip(test_df.Text[spl1], test_pred1[spl1]):
    print sentiment, text

[ 0.13894013  0.86105987] a highly spirited , imaginative kid's movie that broaches neo-augustinian theology : is god stuck in heaven because he's afraid of his best-known creation ?

[ 0.74848078  0.25151922] a work that lacks both a purpose and a strong pulse .

[ 0.76445327  0.23554673] i can analyze this movie in three words : thumbs friggin' down .

[ 0.74280467  0.25719533] a small movie with a big impact .

[ 0.05116639  0.94883361] a comedy that is warm , inviting , and surprising .



**Texto**: ’a’ for creativity but comes across more as a sketch for a full-length comedy .  
**Predicción -1**: 0,96  
**Predicción +1**: 0,04

La opinión es mixta, pero el clasificador la considera más cercana a una opinión negativa.

**Texto**: every once in a while , a movie will come along that turns me into that annoying specimen of humanity that i usually dread encountering the most - the fanboy  
**Predicción -1**: 0,94  
**Predicción +1**: 0,06

La opinión es algo ambigua. Sin embargo, el clasificador la considera más bien una opinión negativa.

**Texto**: it just goes to show , an intelligent person isn’t necessarily an admirable storyteller .  
**Predicción -1**: 0,53  
**Predicción +1**: 0,47

Se ve que la opinión es negativa, pero el clasificador la considera más cercana a una opinión mixta.

**Texto**: the movie is for fans who can’t stop loving anime , and the fanatical excess built into it .  
**Predicción -1**: 0,96  
**Predicción +1**: 0,04

Es una opinión más bien neutra, pero el clasificador la asocia más como una crítica negativa.

**Texto**: there is truth here  
**Predicción -1**: 0,44  
**Predicción +1**: 0,56

Es una opinión cuyo juicio de valor es difícil de determinar, por lo que resulta apropiado que la predicción del clasificador sea mixta.